# Base classes
> Base configuration classes and utilities for the logging library.

In [3]:
#| default_exp client.base

In [ ]:
#| export
from dc_logger.client.Log import LogEntry, LogLevel

In [5]:
#| exporti


from typing import Optional, List, Dict, Any, Literal
from dataclasses import dataclass, field
from abc import ABC, abstractmethod
from enum import Enum



In [6]:
#| export

@dataclass
class LoggerSettings(ABC):
    """abstract base configuration for logging configuration settings"""

    format: str = "json"  # json, text
    batch_size: int = 100
    flush_interval: int = 30  # seconds
    correlation_enabled: bool = True
    include_traceback: bool = True
    max_buffer_size: int = 1000

## Service Handler defines 'route functions' for how to interact with a service.

In [ ]:
#| export


# Type for valid output modes
OutputMode = Literal["cloud", "console", "file", "multi"]


@dataclass
class ServiceConfig(ABC):
    """abstract base class for service-specific configuration settings"""

    output_mode: OutputMode

    def __post_init__(self):
        self.validate_config()


    @abstractmethod
    def validate_config(self) -> bool:
        """Validate the configuration"""
        raise NotImplementedError()


@dataclass
class ServiceHandler(ABC):
    """defines how a handler communicates with services to create logs"""

    service_config: Optional[ServiceConfig] = None # has authentication and connection details to service1

    # @classmethod
    # def from_config(cls, service_config: ServiceConfig):
        
    #     hc = cls(
    #         service_config = service_config
            
    #     )
        
    #     # if hasattr(config, 'to_platform_config') and callable(getattr(config, 'to_platform_config')):
    #     #     hc.platform_config = config.to_platform_config()
    #     return hc
    
    def validate_config(self) -> bool:
        if not self.service_config:
            raise ValueError("Service configuration is not set.")
        
        return self.service_config.validate_config()

    @abstractmethod
    async def write(self, entries: List[LogEntry]) -> bool:
        """Write log entries to destination"""
        pass

    @abstractmethod
    async def flush(self) -> bool:
        """Flush any buffered entries"""
        pass

    async def close(self):
        """Clean up resources"""
        pass


In [ ]:
# | export


@dataclass
class LogHandler(ABC):
    """defines when, how and where to send log entries"""


    log_config: LoggerSettings
    service_handler: ServiceHandler

    handler_name: str = None # friendly name for the handler

    log_level: LogLevel = LogLevel.INFO  # minimum log level to log.

    log_method: List[str] = field(
        default_factory=lambda: ["POST", "PUT", "DELETE", "PATCH"]
    )  # filtered list of API requests to log, generally won't log GET requests

    def __post_init__(self):
        self.validate_config()


    async def validate_config(self) -> bool:
        """Validate the configuration"""

        if not self.service_handler:
            raise ValueError("must set a service handler")
        
        if not self.log_config:
            raise ValueError("must set a log configuration")
        
        is_valid_handler = self.service_handler.validate_config()

        return is_valid_handler
    

    @abstractmethod
    async def write(self, entries: List[LogEntry]) -> bool:
        """Write log entries to destination"""
        raise NotImplementedError()

    @abstractmethod
    async def flush(self) -> bool:
        """Flush any buffered entries"""
        raise NotImplementedError()

    async def close(self):
        """Clean up resources"""
        raise NotImplementedError()

In [ ]:
#| export

@dataclass 
class Logger:
    """ should receive log entries and send them to all handlers.  handlers will use log_level and log_method to determine which logs to send"""
    handlers: List[LogHandler] = field(default_factory=list)
    
    pretty_print: bool = False  # Pretty print JSON for development


    def validate_configs(self) -> bool:
        for handler in self.handlers:
            if not handler.config.validate_config():
                return False
        return True
    
    
    # def get_cloud_config(self) -> Dict[str, Any]:
    #     return {"cloud_provider": "multi"}

    # def get_handler_configs(self) -> List[Dict[str, Any]]:
    #     return [
    #         {
    #             "type": handler.type,
    #             "config": handler.config,
    #             "cloud_config": (
    #                 handler.config.to_platform_config()
    #                 if handler.type == "cloud"
    #                 else None
    #             ),
    #         }
    #         for handler in self.handlers
    #     ]

    # @classmethod
    # def create(
    #     cls,
    #     handlers: List[Dict[str, Any]],
    #     level: LogLevel = LogLevel.INFO,
    #     batch_size: int = 100,
    #     flush_interval: int = 30,
    #     **kwargs
    # ) -> "MultiHandlerLogConfig":
    #     handler_configs = [
    #         HandlerConfig(type=h["type"], config=h["config"]) for h in handlers
    #     ]
    #     return cls(
    #         handlers=handler_configs,
    #         level=level,
    #         batch_size=batch_size,
    #         flush_interval=flush_interval,
    #         **kwargs
    #     )

    

In [12]:
#| hide
import nbdev; nbdev.nbdev_export('./base.ipynb')